<a href="https://colab.research.google.com/github/AnastasiiaLavre/AI4Gov2023/blob/main/2_Data_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 2. In this notebook we split PDFs to individual documents based on the segmentation criteria, merge PDFs and extract the first pages from all individual documents

In [1]:
import pandas as pd
%load_ext google.colab.data_table

In [2]:
#mount google drive to access all the documents
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#to tweak PDFs
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


Merging the PDFs

In [4]:
#merge pdfs
import os
from PyPDF2 import PdfMerger

def merge_pdfs(input_directory, output_file):
    pdf_merger = PdfMerger()

    # Specify the order of filenames to be merged
    desired_order = [
        "40001521.003 0335 - 0435.pdf",
        "40001521.003 0436 - 0535.pdf",
        "40001521.003 0536 - 0635.pdf",
        "40001521.003 0636 - 0668.pdf"
    ]

    for filename in desired_order:
        pdf_path = os.path.join(input_directory, filename)
        pdf_merger.append(pdf_path)

    with open(output_file, "wb") as output_pdf:
        pdf_merger.write(output_pdf)

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/Stanford/Output"  # path to your split PDF directory
    output_merged_file = "/content/drive/MyDrive/Stanford/Output/40001521.003 0335 - 0668.pdf"  # path for the merged PDF output file

    merge_pdfs(input_directory, output_merged_file)

Splitting the documents to individual ones

In [5]:
#split pdf based on "Page" - WORKS for all files in directory
import os
from PyPDF2 import PdfReader, PdfWriter

def process_pdf_files(input_directory, output_directory):
    for filename in os.listdir(input_directory):
        if filename.endswith("0335 - 0668.pdf"):
            input_pdf_file = os.path.join(input_directory, filename)
            file_without_extension = os.path.splitext(filename)[0]

            pdf_reader = PdfReader(input_pdf_file)
            section_number = 1
            current_pdf = PdfWriter()

            for page in pdf_reader.pages:
                text = page.extract_text()

                if "Page" in text or any(f"- {number} -" in text for number in range(50)):
                    current_pdf.add_page(page)
                else:
                    if current_pdf.pages:
                        output_filename = os.path.join(output_directory, f"{file_without_extension}_{section_number}.pdf")
                        with open(output_filename, "wb") as output_file:
                            current_pdf.write(output_file)
                        section_number += 1
                    current_pdf = PdfWriter()
                    current_pdf.add_page(page)

            if current_pdf.pages:
                output_filename = os.path.join(output_directory, f"{file_without_extension}_{section_number}.pdf")
                with open(output_filename, "wb") as output_file:
                    current_pdf.write(output_file)


if __name__ == "__main__":

    input_directory = "/content/drive/MyDrive/Stanford/Output"  # path to the input OCR directory
    output_directory = "/content/drive/MyDrive/Stanford/Output_Split/New"  # path to the split output directory

    process_pdf_files(input_directory, output_directory)

In [ ]:
#40001521.002 0017 - 0334
import os
from PyPDF2 import PdfReader, PdfWriter

def process_pdf_files(input_directory, output_directory):
    for filename in os.listdir(input_directory):
        if filename==("40001521.002 0017 - 0334.pdf"):
            input_pdf_file = os.path.join(input_directory, filename)
            file_without_extension = os.path.splitext(filename)[0]

            pdf_reader = PdfReader(input_pdf_file)
            section_number = 1
            current_pdf = PdfWriter()

            for page in pdf_reader.pages:
                text = page.extract_text()

                if "Page" in text or any(f"- {number} -" in text for number in range(50)):
                    current_pdf.add_page(page)
                else:
                    if current_pdf.pages:
                        output_filename = os.path.join(output_directory, f"{file_without_extension}_{section_number}.pdf")
                        with open(output_filename, "wb") as output_file:
                            current_pdf.write(output_file)
                        section_number += 1
                    current_pdf = PdfWriter()
                    current_pdf.add_page(page)

            if current_pdf.pages:
                output_filename = os.path.join(output_directory, f"{file_without_extension}_{section_number}.pdf")
                with open(output_filename, "wb") as output_file:
                    current_pdf.write(output_file)


if __name__ == "__main__":

    input_directory = "/content/drive/MyDrive/Stanford/Output"  # path to your input OCR directory
    output_directory = "/content/drive/MyDrive/Stanford/Output_Split/"  # path to the split output directory

    process_pdf_files(input_directory, output_directory)



In [ ]:
#data refinement, merge 40001521.002 0017 - 0334_42 and 40001521.002 0017 - 0334_43

import os
from PyPDF2 import PdfMerger

def merge_pdfs(input_directory, output_file):
    pdf_merger = PdfMerger()

    # Specify the order of filenames to be merged
    desired_order = [
        "40001521.002 0017 - 0334_42.pdf",
        "40001521.002 0017 - 0334_43.pdf",
    ]

    for filename in desired_order:
        pdf_path = os.path.join(input_directory, filename)
        pdf_merger.append(pdf_path)

    with open(output_file, "wb") as output_pdf:
        pdf_merger.write(output_pdf)

if __name__ == "__main__":
    input_directory = "/content/drive/MyDrive/Stanford/Output_Split/"  # path to your split PDF directory
    output_merged_file = "/content/drive/MyDrive/Stanford/MergedOutput.pdf"  # path for the merged PDF output file

    merge_pdfs(input_directory, output_merged_file)

Retrieving the first page of each individual document

In [7]:
!pip install PyPDF2
!pip install pytest-shutil
!pip install pdf2image

In [8]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (690 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 120901 

In [9]:
import os
import PyPDF2
from PyPDF2 import PdfReader, PdfWriter
from shutil import copyfile
from PIL import Image
from pdf2image import convert_from_path

In [10]:
# Input directory containing PDF files
input_directory = '/content/drive/MyDrive/Stanford/Output_Split'

# Output directory to save the first pages
output_directory = '/content/drive/MyDrive/Stanford/PDF 1 page'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all PDF files in the input directory
pdf_files = [f for f in os.listdir(input_directory) if f.startswith('40001521.003 0335 - 0668')]

for pdf_file in pdf_files:
    input_path = os.path.join(input_directory, pdf_file)
    output_pdf_path = os.path.join(output_directory, pdf_file)
    output_png_path = os.path.splitext(output_pdf_path)[0] + '.png'

    # Convert the first page to PNG format using pdf2image
    images = convert_from_path(input_path, first_page=0, last_page=1)  # First page is index 0
    if images:
        images[0].save(output_png_path, 'PNG')

    # Save the first page as a separate PDF using PyPDF2
    pdf_writer = PdfWriter()
    pdf_reader = PdfReader(open(input_path, 'rb'))
    first_page = pdf_reader.pages[0]
    pdf_writer.add_page(first_page)
    with open(output_pdf_path, 'wb') as output_pdf:
        pdf_writer.write(output_pdf)

print('First pages of PDF files have been saved to the output directory in both PDF and PNG formats.')


First pages of PDF files have been saved to the output directory in both PDF and PNG formats.
